**Instituto de Informática - UFG**

Disciplina: Inteligência Computacional

Prof. Cedric Luiz de Carvalho

Aluna(o): Arthur Gomes de Siqueira
Ricardo Ataide de Oliveira Junior



---


# Atividade Prática

* Deseja-se identificar a melhor rota para uma viagem rodoviária pelas capitais brasileiras. Para isso pode-se utilizar os dados obtidos do portal: [Itatrans](http://www.itatrans.com.br/distancia.html) e armazenados no arquivo **capitais.csv.**

* A porção inferior da planilha (células abaixo daquela com valor 0), traz à distância rodoviária entre as cidades. A porção superior, indica as distâncias aéreas entre as cidades.

* Considere que o mapa a seguir indique as conexões rodoviárias entre as capitais.  As conexões diretas entre as cidades estão indicadas no arquivo **conexCapitais.csv**

In [ ]:
!mkdir img/

In [ ]:
!mv mapa-brasil-estados-capitais.jpg img/

mv: cannot stat 'mapa-brasil-estados-capitais.jpg': No such file or directory


In [ ]:
from IPython.display import Image
Image("img/mapa-brasil-estados-capitais.jpg")

# Especificação:

  O trabalho deve ser desenvolvido em grupo de dois alunos, conforme especificado pelo professor, em comum acordo com a turma.
  Deve ser entregue um arquivo, no formato de *notebooks Jupyter*, contendo a solução desenvolvida pelo grupo. Esse arquivo deve conter uma descrição detalhada do modelo usado na solução do problema e sua codificação, em Python. Podem ser usadas ideias e implementações de terceiros, desde que sejam feitas as devidas citações das referências utilizadas. O trabalho deve seguir os seguintes passos:

*  Procure identificar soluções para o problema em questão (ou problemas parecidos) desenvolvidas por outras pessoas.
*  Defina um modelo para este problema a partir do estudo dos conceitos e das soluções propostas por outros.
*  Construa uma implementação computacional para a solução do problema, utilizando as seguintes estratégias:

     1.  Busca gulosa 
     2.  Busca A*
     3.  Busca IDA*
     
* Para cada estratégia implementada, meça o tempo gasto para encontrar a solução e o espaço de armazenamento utilizado (número de nós gerados).
* Considere como custo do caminho a distância rodoviária entre as cidades.
* Considere como heurística a distância aérea entre as cidades (distância em linha reta)

* Faça uma avaliação criteriosa da solução produzida pelo grupo.

# Observações Pertinentes:
* Foram enunciados um "Modelo Genérico" e outros "Modelos especificos", onde o modelo genérico trata do modelo base, o qual todas as estratégias de busca possuem em comum, e os modelos especificos serão as particularidades que cada busca possuí.

## Modelo "Genérico" (comum a todas as buscas)

* As capitais serão representadas pelo Data Frame "df_distancia", onde cada conjunto linha x coluna, que esteja "acima" da diagonal de zeros representa a distancia aérea entre a capital "linha x" e a capital "coluna y", e os valores que estejam abaixo da diagonal de zeros, representam a distância rodoviária 

* Também foi definido o dataFrame "df_conexão" , onde cada conjunto linha x coluna representa se existe uma conexão (um caminho direto) a partir da  capital "linha x" para a capital "coluna y"

* Serão utilizados os algoritmos de busca enunciados na especificação do problema para procurar o "melhor" caminho partindo de uma capital até outra qualquer

* O nó raiz da busca será qualquer capital dentre as carregadas no data frame, e o nó objetivo também será qualquer outra capital presente no data frame.



## Implementação

In [ ]:
import pandas as pd
import numpy as np
import time


In [ ]:
#Fazendo upload dos arquivos
from google.colab import files
uploaded = files.upload()

Saving conexCapitais.csv to conexCapitais (1).csv
Saving distCapitais.csv to distCapitais (1).csv


In [ ]:
#Carregando os dados em dataframes
df_conexao = pd.read_csv("conexCapitais.csv")
df_conexao = df_conexao.rename(columns={'Unnamed: 0':'Cidade'}) # O dataframe pode ser carregado utilizando o index como o nome das cidades, ou um número automático. Foi utilizado a abordagem de um index automático
df_conexao = df_conexao.rename(columns={'R. Janeiro':'Rio de Janeiro'})

#Substituindo valores abreviados para facilitar nas buscas
df_conexao = df_conexao.replace({"Cidade": {"B. Horizonte" : "Belo Horizonte"}})
df_conexao = df_conexao.replace({"Cidade": {"C. Grande" : "Campo Grande"}})
df_conexao = df_conexao.replace({"Cidade": {"R. Janeiro": "Rio de Janeiro"}})

#Deletando a coluna e a linha com o nome Macapá, por ser irrelevante no problema
df_conexao = df_conexao.drop(columns="Macapá")
df_conexao = df_conexao.drop([12])
df_conexao = df_conexao.reset_index(drop=True)

#imprime o resultado da tabela conexCapitais.csv
df_conexao

,Cidade,Aracaju,Belém,Belo Horizonte,Boa Vista,Brasília,Campo Grande,Cuiabá,Curitiba,Florianópolis,Fortaleza,Goiânia,João Pessoa,Maceió,Manaus,Natal,Palmas,Porto Alegre,Porto Velho,Recife,Rio Branco,Rio de Janeiro,Salvador,São Luis,São Paulo,Teresina,Vitória
0,Aracaju,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,Belém,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,Belo Horizonte,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
3,Boa Vista,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brasília,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
5,Campo Grande,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
6,Cuiabá,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Curitiba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
8,Florianópolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Fortaleza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN


In [ ]:
#Carregando os dados em dataframes
df_distancia = pd.read_csv("distCapitais.csv")
df_distancia = df_distancia.rename(columns={'Unnamed: 0':'Cidade'})
df_distancia = df_distancia.rename(columns={'R. Janeiro':'Rio de Janeiro'})

## Substituindo nomes abreviados para facilitar nas buscas
df_distancia = df_distancia.replace({"Cidade": {"B. Horizonte" : "Belo Horizonte"}})
df_distancia = df_distancia.replace({"Cidade": {"C. Grande" : "Campo Grande"}})
df_distancia = df_distancia.replace({"Cidade": {"R. Janeiro": "Rio de Janeiro"}})

#Deletando a coluna e a linha com o nome Macapá, por ser irrelevante no problema
df_distancia = df_distancia.drop(columns="Macapá")
df_distancia = df_distancia.drop([12])
df_distancia = df_distancia.reset_index(drop=True)

#imprime o resultado da tabela distCapitais.csv


In [ ]:
df_distancia

,Cidade,Aracaju,Belém,Belo Horizonte,Boa Vista,Brasília,Campo Grande,Cuiabá,Curitiba,Florianópolis,Fortaleza,Goiânia,João Pessoa,Maceió,Manaus,Natal,Palmas,Porto Alegre,Porto Velho,Recife,Rio Branco,Rio de Janeiro,Salvador,São Luis,São Paulo,Teresina,Vitória
0,Aracaju,0,1641,1248,3022,1292,2155,2121,2061,2207,815,1461,486,201,2673,604,1235,2580,2946,398,3359,1482,277,1226,1731,903,1102
1,Belém,2079,0,2111,1432,1592,2212,1778,2665,2904,1133,1693,1636,1680,1292,1550,973,3188,1886,1676,2333,2450,1687,481,2463,750,2275
2,Belo Horizonte,1578,2824,0,3117,624,1118,1372,820,973,1893,666,1726,1439,2556,1831,1178,1341,2477,1639,2786,339,964,1932,489,1652,378
3,Boa Vista,6000,6083,4736,0,2496,2667,2107,3370,3620,2562,2503,3067,3089,661,2983,1988,3785,1335,3103,1626,3428,3009,1913,3300,2169,3394
4,Brasília,1652,2120,716,4275,0,878,873,1081,1314,1687,173,1716,1485,1932,1775,620,1619,1900,1657,2246,933,1060,1524,873,1313,947
5,Campo Grande,2765,2942,1453,3836,1134,0,559,780,1007,2547,705,2593,2352,2013,2654,1320,1119,1634,2530,1827,1212,1905,2284,894,2132,1490
6,Cuiabá,2775,2941,1594,3142,1133,694,0,1302,1543,2329,740,2495,2302,1453,2524,1029,1679,1137,2452,1414,1575,1915,1942,1326,1862,1745
7,Curitiba,2595,3193,1004,4821,1366,991,1679,0,251,2670,972,2545,2259,2734,2645,1693,546,2412,2459,2601,675,1784,2599,338,2362,1076
8,Florianópolis,2892,3500,1301,5128,1673,1298,1986,300,0,2857,1215,2693,2402,2981,2802,1931,376,2641,2603,2809,748,1930,2821,489,2573,1160
9,Fortaleza,1183,1610,2528,6548,2200,3407,3406,3541,3838,0,1854,555,730,2383,435,1300,3213,2855,629,3300,2190,1028,652,2368,495,1855


### Definindo a Heurística

* Como a Heurística foi definida como a distância aérea entre as cidades, foi criada uma função para coletar tal informação a partir do DataFrame.

* Algoritmo desenvolvido pelo grupo

## Caracteristicas

* Conhecimento domínio + problema
* Função avaliação f(n)
* Função heurística h(n)
* Conhecimento na forma de heurísticas

In [ ]:
#Algoritmo uteis para nosso desenvolvimento

def dist_rodoviaria(a,b):
  # o argumento int(df_distancia[df_distancia.Cidade == a].index.values), coleta o valor inteiro do índice da cidade A

  # Se o indicie da cidade A, está acima (possui valor menor) que o indice da cidade b, a distancia rodoviaria, sera indicada no campo df_distancia[indice_B][indice_A]
  if ((int(df_distancia[df_distancia.Cidade == a].index.values)) < (int(df_distancia[df_distancia.Cidade == b].index.values))):
    distancia = df_distancia[df_distancia.Cidade == b].loc[(int(df_distancia[df_distancia.Cidade == b].index.values))][int(df_distancia[df_distancia.Cidade == a].index.values) + 1]
   
    # Se o indicie da cidade A, está acima (possui valor menor) que o indice da cidade b, a distancia rodoviaria, sera indicada no campo df_distancia[indice_B][indice_A]
    #Isso se deve devido a configuração do dataframe df_distancia
  else:
    distancia = df_distancia[df_distancia.Cidade == a].loc[(int(df_distancia[df_distancia.Cidade == a].index.values))][int(df_distancia[df_distancia.Cidade == b].index.values) + 1]

  return float(distancia) 
 
def dist_aerea(a,b):
    #Utiliza o mesmo principio do caso rodoviario, porém é aplicado o "caso reverso"
    if ((int(df_distancia[df_distancia.Cidade == a].index.values)) < (int(df_distancia[df_distancia.Cidade == b].index.values))):
       distancia = df_distancia[df_distancia.Cidade == a].loc[(int(df_distancia[df_distancia.Cidade == a].index.values))][int(df_distancia[df_distancia.Cidade == b].index.values) + 1]
   
    else:
      distancia = df_distancia[df_distancia.Cidade == b].loc[(int(df_distancia[df_distancia.Cidade == b].index.values))][int(df_distancia[df_distancia.Cidade == a].index.values) + 1]
  
    return float(distancia)

#vai marcar o tempo
def marcar_tempo():
  return time.time();

In [ ]:
# Função que irá retornar os vizinhos/possíveis filhos de um dado nó que estamos atualmente
def vizinhos(cidade_atual):
  conexao_vizinhos = df_conexao[cidade_atual] #conexao da cidade_atual com outras cidades vizinhos
  lista_dos_vizinhos = []
  
  for indice, possui_conexao in enumerate(conexao_vizinhos):
      if (possui_conexao == 1.0): #se for igual 1.0, quer dizer que possui conexao
        cidade_vizinha = df_conexao.Cidade[indice] #pega a cidade
        
        if (cidade_vizinha != cidade_atual): #a cidade vizinha deve ser diferente dela mesmo
          lista_dos_vizinhos.append(cidade_vizinha) #adiciona como vizinho
  
  return lista_dos_vizinhos

#um simples teste para verificar que esta funcionando a função vizinhos
print(vizinhos("São Paulo"))


['Belo Horizonte', 'Brasília', 'Campo Grande', 'Curitiba', 'Goiânia', 'Rio de Janeiro']


### Busca Gulosa

#### Modelo da Busca:

* O Algoritmo irá expandir o nó que possui o menor valor "heurístico", dado pela função heurística(neste caso, definida pela função `dist_aerea()`) 

* f(n) = h(n) (heurística) = estimativa do custo de n até o objetivo

* Desta maneira, no nó raiz serão expandidos todos os nós, filhos da raiz, posteriormente será aplicada a função heurística a cada nó expandido, e, será expandido o que possuir o menor valor da função heurística.

* O método não é completo e nem ótimo, e se "assemelha" um pouco ao de busca em profundidade, no entanto ao invés do nó expandido ser o primeiro gerado, a expansão se dará pela analise da função heurística. No entanto, a função heurística garante a possibilidade de resultados muito mais interessantes que uma busca cega.

#### Analise

* Completa: Não, pois pode ficar presa em loops,
* Tempo: O(b^m) no pior caso, mas uma boa função heurística pode levar a uma redução substancial, m é a profundidade máxima do espaço de busca
* Espaço: O(b^m) – mantém todos os nós na memória, onde m é a profundidade máxima do espaço de busca
* Ótima: Não é ótima, pois segue o melhor passo considerando somente o estado atual.

In [ ]:
#Algoritmo Guloso - Algoritmo desenvolvido pelo grupo com base na definição do teórica do algoritmo
def busca_gulosa(cidade_atual,destino):

  #Algoritmo Guloso Recursivo
  def busca_gulosa_recursiva(cidade_atual, destino, caminho, distancia):
    if cidade_atual == destino: #Acontece quando chegamos ao nosso destino  
      caminho.append(cidade_atual) #Adicionamos nossa cidade atual
      caminho.append(distancia) # mais a distancia
      return caminho
    else:
      caminho.append(cidade_atual) #adiciona a cidade no caminho
      vizinhos_da_cidade_atual = vizinhos(cidade_atual) #pegamos os vizinhos da cidade
      
      #percorre todas as cidades vizinhas da cidade atual
      for indice, viz in enumerate(vizinhos_da_cidade_atual):
        distancia_viz = float(dist_aerea(viz,destino)) #pega a distancia aerea
        if indice == 0: #quando é o primeiro vertice da um set nas variaveis
          menor_heuristica = distancia_viz #pega a menor distancia aerea
          proxima_cidade = viz #pega a proxima cidade
        else:
          if (distancia_viz < menor_heuristica): #se tiver uma distancia aerea mais curta que a menor
            menor_heuristica = distancia_viz #pega a menor distancia aerea
            proxima_cidade = viz #pega a proxima cidade

      distancia = distancia + float(dist_rodoviaria(cidade_atual,proxima_cidade)) #soma a distancia que ja possuimos com a distancia rodoviaria que proxima cidade
      busca_gulosa_recursiva(proxima_cidade,destino,caminho,distancia) #executa a funçao de novo so que com a proxima cidade excolhida

  caminho_final = [] #variavel com todo o caminho
  busca_gulosa_recursiva(cidade_atual,destino,caminho_final,0)
  
  print("Distancia: ", caminho_final.pop()) #Resultado da distancia 
  print("Caminho: ", caminho_final) #Resultado do caminho

In [ ]:
busca_gulosa("São Paulo", "Manaus")

Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Cuiabá', 'Porto Velho', 'Manaus']


### Busca A*

#### Modelo de Busca:

##### Descrição genérica do algorimo:

* A busca A* considera um função mais "rigorosa" que a busca gulosa
* Primeiramente, serão expandidos todos os nós "filhos" do nó raiz
* Após expandidos será aplicada uma função que podemos chamar de estimativa de custo, a qual é definida por `f(n) = h(n) + g(n)`, onde, `g(n)` é o custo "real" do nó raiz até o nó que está sendo analisado, e, `h(n)` é a própria função heurística. 

##### Análise
* Completa: Sim
* Tempo: é exponencial no erro máximo absoluto, ou seja, O(b^d),
onde d é a solução de profundidade.
* Espaço: Mantém todos os nós na memória
* Ótima: Sim e consegue ser provada com heuristica admissivel e consistente

In [ ]:
# Algoritmo busca A* - Algoritmo desenvolvido pelo grupo com base na definição do teórica do algoritmo
def busca_a_estrela(cidade_atual,destino):

  def busca_a_estrela_recursiva(cidade_atual, destino):
    caminho = [] #declara uma variavel do tipo set
    fila = [] #cria uma lista -> distancia_heuristica, distancia rodoviaria, cidade atual e caminho da origem ao destino
    fila.append([0, 0, cidade_atual, caminho]) #adiciona a cidade selecionada como primeira da fila

    while len(fila) != 0:
      distancia_heuristica, distancia, cidade, caminho_percorrido = fila.pop(0) #pega a distancia já calculado e a cidade onde estamos atualmente 
      if(cidade == destino): #achou a cidade destino
        caminho_percorrido.append(cidade) #adiciona cidade no caminho
        return [distancia, caminho_percorrido]

      if cidade not in caminho_percorrido: #cidade nao visitada ainda
        caminho_percorrido.append(cidade)
        
        for i, conexao in enumerate(df_conexao[cidade]): 
          if conexao == 1: #verifica se ha conexao
            cidade_viz = df_distancia.Cidade[i] #pega a cidade vizinha
            
            if cidade_viz in caminho: #cidade vizinha ja visitada
              continue
            
            distancia_aerea = dist_aerea(cidade_viz,destino) #pega a distancia aerea
            distancia_rodoviaria = dist_rodoviaria(cidade,cidade_viz) #pega a distancia rodoviaria
            fila.append([distancia_aerea + distancia_rodoviaria, distancia + distancia_rodoviaria, cidade_viz, caminho_percorrido]) #faz a conta da funçao heuristica e adiciona ela

      fila.sort(key=lambda x:x[0]) #ordena a fila, sempre queremos a menor distancia possivel
      
  resultado_final = busca_a_estrela_recursiva(cidade_atual,destino) #resultado final com a distancia e o caminho
  print("Distancia: ", resultado_final[0]) #distancia
  print("Caminho: ", resultado_final[1]) # caminho


In [ ]:
busca_a_estrela("São Paulo", "Manaus")

Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Brasília', 'Cuiabá', 'Porto Velho', 'Manaus']


### Busca IDA*



#### Modelo de Busca:

##### Descrição genérica do algorimo:
* Estratégia geral usada com frequência em conjunto com busca em profundidade
* Encontra o melhor limite de profundidade

##### Análise

* Completa: Sim, quando fator de ramificação é finito
* Tempo: Complexidade de tempo O(b^d), onde d é a solução de profundidade.
* Espaço: O(bd), onde d é a solução de profundidade.
* Ótima: Sim, quando custo de caminho cresce com a profundidade do nó

In [ ]:
# Algoritmo Busca IDA* - Algoritmo desenvolvido pelo grupo com base na definição do teórica do algoritmo
def busca_ida_estrela(cidade_atual,destino, limite):

  def busca_ida_estrela_recursiva(cidade_atual, destino, limite):
    caminho = [] #declara uma variavel do tipo set
    fila = [] #cria uma lista -> distancia_heuristica, distancia rodoviaria, cidade atual e caminho da origem ao destino
    fila.append([0, 0, cidade_atual, caminho]) #adiciona a cidade selecionada como primeira da fila

    while len(fila) != 0:
      distancia_heuristica, distancia, cidade, caminho_percorrido = fila.pop(0) #pega a distancia já calculado e a cidade onde estamos atualmente 
      
      if(len(caminho_percorrido) > limite): #verifica se é maior que o limite
        return [-1, []]
      
      if(cidade == destino): #achou a cidade destino
        caminho_percorrido.append(cidade)
        return [distancia, caminho_percorrido]

      if cidade not in caminho_percorrido: #cidade nao visitada ainda
        caminho_percorrido.append(cidade)
        
        for i, conexao in enumerate(df_conexao[cidade]): 
          if conexao == 1: #verifica se ha conexao
            cidade_viz = df_distancia.Cidade[i]
            
            if cidade_viz in caminho: #cidade vizinha ja visitada
              continue
            
            distancia_aerea = dist_aerea(cidade_viz,destino) #pega a distancia aerea
            distancia_rodoviaria = dist_rodoviaria(cidade,cidade_viz) #pega a distancia rodoviaria
            fila.append([distancia_aerea + distancia_rodoviaria, distancia + distancia_rodoviaria, cidade_viz, caminho_percorrido]) #faz a conta da funçao heuristica e adiciona ela

      fila.sort(key=lambda x:x[0]) #ordena a fila, sempre queremos a menor distancia possivel
      
  resultado_final = busca_ida_estrela_recursiva(cidade_atual, destino, limite) #execução da função
  if(resultado_final[0] != -1):  #se o limite for mais alto
    print("Distancia: ", resultado_final[0]) #distancia
    print("Caminho: ", resultado_final[1]) #caminho
  else:
    print("Caminho passou do limite")

In [ ]:
busca_ida_estrela("São Paulo", "Manaus", 10)

Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Brasília', 'Cuiabá', 'Porto Velho', 'Manaus']


## Testes

Nessa seção iremos fazer testes com todos os algoritmos mencionados anteriormente.

In [ ]:
#1 Teste de São Paulo até Manaus
print("1 Teste de São Paulo até Manaus\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("São Paulo", "Manaus")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("São Paulo", "Manaus")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("São Paulo", "Manaus", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#2 Teste de São Paulo até Rio de Janeiro
print("2 Teste de São Paulo até Rio de Janeiro\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("São Paulo", "Rio de Janeiro")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("São Paulo", "Rio de Janeiro")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("São Paulo", "Rio de Janeiro", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#3 Teste de Belo Horizonte até Recife
print("3 Teste de Belo Horizonte até Recife\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("Belo Horizonte", "Recife")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("Belo Horizonte", "Recife")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("Belo Horizonte", "Recife", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#4 Teste de Porto Alegre até Teresina
print("4 Teste de Porto Alegre até Teresina\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("Porto Alegre", "Teresina")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("Porto Alegre", "Teresina")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("Porto Alegre", "Teresina", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#5 Teste de Goiânia até Rio Branco
print("5 Teste de Goiânia até Rio Branco\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("Goiânia", "Rio Branco")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("Goiânia", "Rio Branco")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("Goiânia", "Rio Branco", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#6 Teste de João Pessoa até Belém
print("6 Teste de João Pessoa até Belém\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("João Pessoa", "Belém")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("João Pessoa", "Belém")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("João Pessoa", "Belém", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)

print("\n\n")

#7 Teste de Rio de Janeiro até Brasília
print("7 Teste de Rio de Janeiro até Brasília\n")
print("Busca Gulosa")
tempo_inicial = marcar_tempo()
busca_gulosa("Rio de Janeiro", "Brasília")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca A-estrela")
tempo_inicial = marcar_tempo()
busca_a_estrela("Rio de Janeiro", "Brasília")
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("----")
print("Busca IDA-estrela")
tempo_inicial = marcar_tempo()
busca_ida_estrela("Rio de Janeiro", "Brasília", 10)
tempo_final = marcar_tempo()
print('O tempo percorrido: ', tempo_final - tempo_inicial)
print("\n\n")

1 Teste de São Paulo até Manaus

Busca Gulosa
Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Cuiabá', 'Porto Velho', 'Manaus']
O tempo percorrido:  0.04402279853820801
----
Busca A-estrela
Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Brasília', 'Cuiabá', 'Porto Velho', 'Manaus']
O tempo percorrido:  0.07433247566223145
----
Busca IDA-estrela
Distancia:  4217.0
Caminho:  ['São Paulo', 'Goiânia', 'Brasília', 'Cuiabá', 'Porto Velho', 'Manaus']
O tempo percorrido:  0.0734555721282959



2 Teste de São Paulo até Rio de Janeiro

Busca Gulosa
Distancia:  429.0
Caminho:  ['São Paulo', 'Rio de Janeiro']
O tempo percorrido:  0.014369487762451172
----
Busca A-estrela
Distancia:  429.0
Caminho:  ['São Paulo', 'Rio de Janeiro']
O tempo percorrido:  0.026024580001831055
----
Busca IDA-estrela
Distancia:  429.0
Caminho:  ['São Paulo', 'Rio de Janeiro']
O tempo percorrido:  0.02406167984008789



3 Teste de Belo Horizonte até Recife

Busca Gulosa
Distancia:  3097.0
Caminho:  ['Belo H

# Comparaçao com o Google Maps (selecionado somente os menores caminhos)

* São Paulo - Manaus: 3875 km
* São Paulo - Rio de janeiro: 435 Km
* Belo Horizonte - Recife: 2026 Km
* Porto Alegre - Teresina: 3779 Km
* Goiânia - Rio Branco: 2847 Km
* João Pessoa - Belém: 2057 Km
* Rio de Janeiro - Brasília: 1166 Km






In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
#Função para gerar um dataFrame com as informações de cada um dos testes, o qual será utilizado para comparação dos resultados
def teste(google,gulosa, a_estrela,ida_estrela):
  df_teste = {'Distancia': [google,gulosa,a_estrela,ida_estrela]}
  df_teste = pd.DataFrame(df_teste, index = ["GOOGLE","GULOSA","A_ESTRELA","IDA_ESTRELA"])
  return df_teste
  
df_teste1 = teste(3875,4217,4217,4217)
df_teste2 = teste(435,429,429,429)
df_teste3 = teste(2026,3097,3092,3092)
df_teste4 = teste(3779,5569,4499,0)
df_teste5 = teste(2847,4736,4736,4736)
df_teste6 = teste(2057,2608,2608,2608)
df_teste7 = teste(1166,1150,1150,1150)



## Comparação dos resultados

#### Teste 1 - São Paulo - Manaus

In [ ]:
#Gerando gráficos com o plotly
 fig = px.bar(df_teste1, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="São Paulo - Manaus")  
 fig.show( )


#### Teste 2 - São Paulo - Rio de Janeiro

In [ ]:
 fig = px.bar(df_teste2, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="São Paulo - Rio de Janeiro")  
 fig.show( )

#### Teste 3 - Belo Horizonte - Recife

In [ ]:
 fig = px.bar(df_teste3, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="Belo Horizonte - Recife")  
 fig.show()

#### Teste 4 - Porto Alegre - Teresina

In [ ]:
 fig = px.bar(df_teste4, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="Porto Alegre - Teresina")  
 fig.show()

#### Teste 5 - Goiânia - Rio Branco

In [ ]:
 fig = px.bar(df_teste5, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="Goiânia - Rio Branco")  
 fig.show()

#### Teste 6 - João Pessoa - Belém

In [ ]:
 fig = px.bar(df_teste6, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="João Pessoa - Belém")  
 fig.show()

#### Teste 7 - Rio de Janeiro - Brasília

In [ ]:
 fig = px.bar(df_teste7, y = "Distancia", x=df_teste1.index.values, color=df_teste1.index.values, title="Rio de Janeiro - Brasília")  
 fig.show()


## Analise dos Resultados

Com os resultados de cada uma das soluções em mãos é fácil notar que técnica utilizada pelo Google é muito superior ao modelo aqui implementado, no entanto considerando que nosso modelo é extremamente limitado e, simplificado, tal resultado é até esperado.

Ao considerarmos então, que possuímos um modelo bem limitado, baseado nos dados que recebemos, os quais realmente não consideram inúmeras variáveis da realidade, todas as estratégias de busca apresentaram resultados muito similares, inclusive, é possível verificar que embora a busca gulosa seja a mais limitada, na maioria dos testes realizados, estea apresentou o resultado ótimo, o que mostra a grande efiência da estratégia para problemas não tão complexos, com poucos nós de profundidade, o que é o caso que estamos lidando. 

Desta maneira, é possível notar que o resultado com a inserção das funções heurísticas é extremamente mais eficaz, do que o modelo abordado anteriormente, de busca cegas. No entanto, ao comparar com as melhores solucões do mercado, o nosso modelo ainda é muito limitado, até mesmo para que seja possível realizar uma comparação "justa".